In [1]:
import numpy as np
import scipy.sparse as sp
import numba

# Problem Definition

## Input

Given two $N \times N$ sparse marices $\mathbf{A}$ and $\mathbf{B}$:

* A is a diagonal with only non-zeros at the main diagonal. 
  * In diagonal format, https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dia_matrix.html
* B is a normal matrix with approximately 4 non-zeros per row, randomly distributed.
  * In CSR format, https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

## Target

How to efficiently calculate the matrix product $\mathbf{C} = \mathbf{A} \mathbf{B}$?

* The result matrix should also be in CSR format.
* The matrix should have canonical format
  * Column indices are sorted
  * No duplicated entries


In [2]:
N = int(1e6)
np.random.seed(1)

A = sp.dia_matrix((np.random.randn(1, N), [0]), shape=(N, N))
B = sp.csr_matrix(
    (
        np.random.randn(N * 4),  # 4N elements
        (
            np.repeat(np.arange(N), 4),
            np.random.randint(low=N, high=None, size=N * 4)
        )
    ),
    shape=(N, N)
)
B.sort_indices()
B.sum_duplicates()

# Naive Calculation

Call sparse matrix multiply using `scipy.sparse`.

In [3]:
def naive_multiply(x, y):
    z = x * y
    z.sort_indices()
    z.sum_duplicates()
    return z

In [4]:
C1 = naive_multiply(A, B)

In [5]:
%timeit naive_multiply(A, B)

153 ms ± 3.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Direct Manipulation of Sparse Matrix

The result of the right-multiplication of a diagonal matrix to a regular matrix.

$
\begin{bmatrix}
a_1    & 0        & \cdots & 0      \\
0      & a_2      & \cdots & 0      \\
\vdots & \vdots   & \ddots & \vdots \\
0      & \vdots   & \cdots & a_N   
\end{bmatrix}
\begin{bmatrix}
b_{11} & b_{12} & \cdots   & b_{1N} \\
b_{21} & b_{22} & \cdots   & b_{2N} \\
\vdots & \vdots & \ddots   & \vdots \\
b_{N1} & b_{N2} & \cdots   & b_{NN}
\end{bmatrix}
=
\begin{bmatrix}
a_1 b_{11} & a_1 b_{12} & \cdots   & a_1 b_{1N} \\
a_2 b_{21} & a_2 b_{22} & \cdots   & a_2 b_{2N} \\
\vdots     & \vdots     & \ddots   & \vdots \\
a_N b_{N1} & a_N b_{N2} & \cdots   & a_N b_{NN}
\end{bmatrix}
$

Note $\mathbf{B}$ is a sparse matrix in CSR format, the resulted $\mathbf{C}$ will have exactly the same sparse structure as $\mathbf{B}$.
* The structure of $\mathbf{B}$ can be directly copied to $\mathbf{C}$
* The data of $\mathbf{C}$ is the multiplication of the data in $\mathbf{B}$ and the corresponding values in $\mathbf{A}$.

## Diagonal Matrix

In diagonal matrix `A` there is a property `data`. It is a `1*N` array containing the diagonal entries of $\mathbf{A}$.

## CSR Matrix

In CSR matrix `B` there are following properties:

* `nnz`: number of non-zeros
* `data`: a flat array containing all the non-zeros in the matrix, ordered by row. That's why it is allced compressed sparse row.
* `indices`: an integer array of the column indices of the data
* `indptr`: an `nrow + 1` array specifying the row composition. The column indices for row `i` are stored in `indices[indptr[i]:indptr[i+1]]` and their corresponding values are stored in `data[indptr[i]:indptr[i+1]]`

### Example

$
\begin{bmatrix}
1 & 0 & 5 & 0 \\
0 & 0 & 0 & 7 \\
3 & 4 & 6 & 0 \\
0 & 1 & 0 & 0 \\
\end{bmatrix}
$

```python
nnz = 7
data = [1, 5, 7, 3, 4, 6, 1]
indices = [0, 2, 3, 0, 1, 2, 1]
indptr = [0, 2, 3, 6, 7]
```

### Direct Manipulation 

To calculate `C`:

* the structure of `B` (`indices`, `indptr`) can be directly copied.
* the `data` of `C` has excatly the same length as `data` of `B`, the data is calculated as follows
  * for a given row i, `C.data[indptr[i]:indptr[i + 1]] = B.data[indptr[i]:indptr[i + 1]] * A.data.ravel()[i]`

# Python Implementation of the direct manipulation

It can be very slow due to Python intepreter

In [6]:
def multiply_in_python(x, y):
    res = y.copy()
    indptr = res.indptr
    for i in range(y.shape[0]):
        res.data[indptr[i]:indptr[i + 1]] *= x.data.ravel()[i]
    return res

In [7]:
C2 = multiply_in_python(A, B)
np.all(C1.data == C2.data)

True

In [8]:
%timeit multiply_in_python(A, B)

2.06 s ± 35.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Vectorization using Numpy

Try to calculate `C.data` in one-go. The `A.data` has to be expaned with exactly the same length as `B.data`. For row `i`, if there are 4 nonzeros in `B`, the entry in `A.data` needs to be repeated 4 times.

## Example

$
\mathbf{A} = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 2 & 0 & 0 \\
0 & 0 & 3 & 0 \\
0 & 0 & 0 & 4 \\
\end{bmatrix}
$

$
\mathbf{B} = \begin{bmatrix}
1 & 0 & 5 & 0 \\
0 & 0 & 0 & 7 \\
3 & 4 & 6 & 0 \\
0 & 1 & 0 & 0 \\
\end{bmatrix}
$


$
\mathbf{C} = \mathbf{A} \mathbf{B} = \begin{bmatrix}
1 & 0  & 5  & 0 \\
0 & 0  & 0  & 14 \\
9 & 12 & 18 & 0 \\
0 & 4  & 0  & 0 \\
\end{bmatrix}
$

```python
A.data = [[1, 2, 3, 4]]
A.data.ravel() = [1, 2, 3, 4]

B.nnz = 7
B.data = [1, 5, 7, 3, 4, 6, 1]
B.indices = [0, 2, 3, 0, 1, 2, 1]
B.indptr = [0, 2, 3, 6, 7]

C = B.copy()
ncols_per_row = np.diff(B.indptr) = [2, 1, 3, 1]
row_indices = np.repeat(np.arange(B.shape[0]), ncols_per_row) = [0, 0, 1, 2, 2, 2, 3]
A.data.ravel()[row_indices] = [1, 1, 2, 3, 3, 3, 4]
C.data *= A.data.ravel()[row_indices] = [1, 5, 14, 9, 12, 18, 4]
```

In [9]:
def multiply_in_numpy(x, y):
    res = y.copy()
    ncols_per_row = np.diff(y.indptr)
    row_indices = np.repeat(np.arange(y.shape[0]), ncols_per_row)
    res.data *= x.data.ravel()[row_indices]
    return res


In [10]:
C3 = multiply_in_numpy(A, B)
np.all(C1.data == C3.data)

True

In [11]:
%timeit multiply_in_numpy(A, B)

32.2 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# JIT (numba) Implementation of the Direct Manipulation

* Numpy implementation needs many intermediate arrays, which is time consuming
* Write the original for-loop in compiled language, for example, numba JIT. Only one loop is needed

In [12]:
@numba.njit
def numba_core(indptr, data_x, data_y, data_z):
    size = data_x.shape[0]
    for i in range(size):
        for k in range(indptr[i], indptr[i + 1]):
            data_z[k] = data_x[i] * data_y[k]

def multiply_in_numba(x, y):
    z_data = np.empty_like(y.data)
    numba_core(y.indptr, x.data.ravel(), y.data, z_data)
    return sp.csr_matrix((z_data, y.indices, y.indptr), shape=(N, N))

C4 = multiply_in_numba(A, B)

In [13]:
np.all(C1.data == C4.data)

True

In [14]:
%timeit multiply_in_numba(A, B)

7.09 ms ± 83.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Total Comparison

In [15]:
print('Naive Calculation')
%timeit naive_multiply(A, B)
print('Direct Manipulation in Python')
%timeit multiply_in_python(A, B)
print('Vectorization in Numpy')
%timeit multiply_in_numpy(A, B)
print('Direct Manipulation in numba')
%timeit multiply_in_numba(A, B)

Naive Calculation
159 ms ± 3.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Direct Manipulation in Python
2.07 s ± 54.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Vectorization in Numpy
33.6 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Direct Manipulation in numba
8.61 ms ± 575 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
